In [ ]:
%pip install transformers==4.40.1 peft==0.5.0
%pip install sentencepiece
%pip install accelerate
%pip install torch
%pip install peft
%pip install datasets
%pip install bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.0/138.0 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 83.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.6/85.6 kB 8.8 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.44.2
    Uninstalling transformers-4.44.2:
      Successfully uninstalled transformers-4.44.2
  Attempting uninstall: peft
    Found existing installation: peft 0.13.2
    Uninstalling peft-0.13.2:
      Successfully uninstalled peft-0.13.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 3.2.1 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.40.1 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 

In [ ]:
from huggingface_hub import login
login("YOUR_USERNAME", use_keyring=True)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
from transformers import AutoModel, AutoTokenizer, AutoModelForCausalLM, LlamaForCausalLM, LlamaTokenizerFast
from peft import PeftModel
import torch
import pandas as pd

In [ ]:
# Define model and tokenizer with authentication token
base_model = "meta-llama/Meta-Llama-3-8B"
peft_model = "FinGPT/fingpt-mt_llama3-8b_lora"
token = "YOUR_TOKEN"

In [ ]:
# Set device to CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [ ]:
# Load the tokenizer
tokenizer = LlamaTokenizerFast.from_pretrained(base_model, use_auth_token=token, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

# Load the base model and the LoRA fine-tuned model on CPU
model = LlamaForCausalLM.from_pretrained(
    base_model,
    use_auth_token=token,
    trust_remote_code=True,
    torch_dtype=torch.float16
)
model = model.to(device)
model = PeftModel.from_pretrained(model, peft_model)
model = model.to(device)
model = model.eval()

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1958: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'PreTrainedTokenizerFast'. 
The class this function is called from is 'LlamaTokenizerFast'.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:2991: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.


...

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/177 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


adapter_config.json:   0%|          | 0.00/650 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/13.6M [00:00<?, ?B/s]

In [ ]:
# Read the financial statements from PLYA_sheet.txt
with open("PLYA_sheet.txt", "r") as file:
    financial_statements = file.read()

In [ ]:
# Read the CSV file
financial_statements_df = pd.read_csv("balance_sheet.csv")

In [ ]:
financial_statements_df

,label,2023,2022
0,ASSETS,NaN,NaN
1,Cash and cash equivalents,2.725200e+08,2.839450e+08
2,"Trade and other receivables, net",NaN,NaN
3,Related Party,5.861000e+06,NaN
4,Nonrelated Party,7.476200e+07,NaN
5,Insurance recoverable,9.821000e+06,3.419100e+07
6,Inventories,1.996300e+07,2.004600e+07
7,Prepayments and other assets,5.429400e+07,4.417700e+07
8,"Property and equipment, net",1.415572e+09,1.536567e+09
9,Derivative financial instruments,2.966000e+06,3.510000e+06


In [ ]:
# prompt = f"""
# Extract the following key financial values from the company's balance sheet for 2023:
# 1. Revenue
# 2. Net Income
# 3. Operating Income
# 4. Total Assets
# 5. Shareholders' Equity
# 6. Cost of Goods Sold
# 7. Current Assets
# 8. Current Liabilities
# 9. Inventory
# 10. Cash and Cash Equivalents
# 11. Total Debt
# 12. EBIT (Earnings Before Interest and Taxes)
# 13. Interest Expense
# 14. Operating Cash Flow
# 15. Capital Expenditures
# 16. Dividends

# Please list each value in the format:
# - Revenue: [value]
# - Net Income: [value]
# ...

# {financial_statements}

# Do not repeat the prompt. Provide only the calculated ratios and analysis.
# """

prompt = f"""analyze the sentiment
{financial_statements_df}
Extract key financial values such as revenue and net income
"""

In [ ]:
# Tokenize and encode the prompt
tokens = tokenizer(prompt, return_tensors="pt", padding=True, max_length=512).to(device)

# Generate the calculated ratios and analysis using FinGPT
with torch.no_grad():
    res = model.generate(**tokens, max_new_tokens=1024)#, temperature=0.2, repetition_penalty=1.5)

# Decode the output tokens into text
calculated_ratios_and_analysis = tokenizer.decode(res[0])#, skip_special_tokens=True)


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2692: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [ ]:
# Print the results
print("Financial Ratios and Analysis:")
print(calculated_ratios_and_analysis)


Financial Ratios and Analysis:
<|begin_of_text|>analyze the sentiment
                                                label          2023  \
0                                              ASSETS           NaN   
1                           Cash and cash equivalents  2.725200e+08   
2                    Trade and other receivables, net           NaN   
3                                       Related Party  5.861000e+06   
4                                    Nonrelated Party  7.476200e+07   
5                               Insurance recoverable  9.821000e+06   
6                                         Inventories  1.996300e+07   
7                        Prepayments and other assets  5.429400e+07   
8                         Property and equipment, net  1.415572e+09   
9                    Derivative financial instruments  2.966000e+06   
10                                      Goodwill, net  6.064200e+07   
11                            Other intangible assets  4.357000e+06   
12     